# Twitter crawler

Adapted from: [vickyqian/twitter crawler.txt](https://gist.github.com/vickyqian/f70e9ab3910c7c290d9d715491cde44c)
And help from: [30 Days of Python](https://www.youtube.com/watch?v=dvAurfBB6Jk)

In [1]:
import tweepy
import webbrowser
import wget
import ast
import csv
import pandas as pd
####input your credentials here
consumer_key = '9qaIUNkefTsh7aakUtDGEYcgf'
consumer_secret = 'kXyhIyrZpAMQrZ5z9ZS8zhxdSrIkcVuWZBgvASYamHWhyn7Ymn'
callback_url = 'oob'

In [2]:
#authorization
auth = tweepy.OAuthHandler(consumer_key, consumer_secret,callback_url)
redirect_url = auth.get_authorization_url()
api = tweepy.API(auth,wait_on_rate_limit=True)
webbrowser.open(redirect_url)
user_pin_input= input("What's the pin value?")
auth.get_access_token(user_pin_input)
api = tweepy.API(auth)

What's the pin value? 4576656


In [17]:
#####Selfies
# Open/Create a file to append data
csvFile = open('nomask_tweets.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,q="#nomaskselfie" + "-filter:retweets",count=100,
                           lang="en", include_entities = True).items():
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8'),tweet.entities,tweet.geo])

# Dataframe

In [18]:
df = pd.read_csv('nomask_tweets.csv')
                 
df.columns = ['created','text','entities','geo']

In [19]:
df

,created,text,entities,geo
0,2020-08-05 19:33:53,b'@MayorofLondon Everything going on in the wo...,"{'hashtags': [{'text': 'NoNewNornal', 'indices...",NaN
1,2020-08-05 18:49:18,b'Speak the word.\xf0\x9f\x91\x86\xf0\x9f\x91\...,{'hashtags': [{'text': 'HowToKnowIfYouAreGoodI...,NaN
2,2020-08-05 18:07:29,"b""I think Aldi are taking the piss with traffi...","{'hashtags': [{'text': 'KBF', 'indices': [94, ...",NaN
3,2020-08-05 17:51:04,"b'This is about your leader, Simon Dolan, the ...","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
4,2020-08-05 15:26:24,"b""Forgot the milk yesterday so back to the con...","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN
...,...,...,...,...
271,2020-07-28 05:44:48,b'@BreesAnna This is it! If the blind won\xe2\...,"{'hashtags': [{'text': 'masks', 'indices': [83...",NaN
272,2020-07-28 03:28:33,b'@maxH6294 Why does almost everyone taking a ...,"{'hashtags': [{'text': 'NoMaskSelfie', 'indice...",NaN
273,2020-07-28 03:23:15,b'Why does almost everyone taking a #NoMaskSel...,"{'hashtags': [{'text': 'NoMaskSelfie', 'indice...",NaN
274,2020-07-28 00:29:14,b'This #NoMaskSelfie is a load of bollocks. Th...,"{'hashtags': [{'text': 'NoMaskSelfie', 'indice...",NaN


In [20]:
df['entities'][10]

"{'hashtags': [{'text': 'kbf', 'indices': [100, 104]}, {'text': 'nomask', 'indices': [105, 112]}, {'text': 'nomaskselfie', 'indices': [113, 126]}, {'text': 'nomaskonme', 'indices': [128, 139]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/hHdLtnTCET', 'expanded_url': 'https://www.bbc.co.uk/news/uk-wales-53653235', 'display_url': 'bbc.co.uk/news/uk-wales-…', 'indices': [75, 98]}]}"

# Get media

In [21]:
media_files = set()
for item in df.entities:
    i = ast.literal_eval(item)
    media = i.get('media', [])
    if(len(media) > 0):
        media_files.add(media[0]['media_url'])
len(media_files)

35

In [22]:
for media_file in media_files:
    wget.download(media_file, out='nomask_images')